In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import tqdm
import re
import string
import nltk
from nltk.corpus import stopwords
import shutil

from argparse import Namespace
arg = Namespace()

import sys
import os

if os.path.abspath("../src") not in sys.path:
    sys.path.append(os.path.abspath("../src"))

import preprocessing as pp
import config
import importlib

importlib.reload(pp)

<module 'preprocessing' from 'c:\\Users\\Gus\\Documents\\proyectos\\REST-MEX-2025\\src\\preprocessing.py'>

In [2]:
train_set = pd.read_csv(config.TRAIN_FILE)
train_set.head()

,Title,Review,Polarity,Town,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",4.0,Isla_Mujeres,QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel


In [35]:
X = train_set.drop(columns=config.TARGETS)
y1 = train_set[config.TARGET1] # Polarity
y2 = train_set[config.TARGET2] # Town
y3 = train_set[config.TARGET3] # Type

X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42, stratify=y1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42, stratify=y2)
X3_train, X3_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42, stratify=y3)

### Preprocesado

In [36]:
def preproccess_pipeline():
    """
    This function defines a preprocessing pipeline for text data.

    Steps in the pipeline:
    1. 'Arreglar mojibakes': Fixes mojibake issues in the specified text columns.
    2. 'Minúsculas y quitar stopwords': Converts text to lowercase and removes stopwords from the specified text columns.
    3. 'Quitar features no deseadas': Drops unwanted features from the specified text columns.

    Args:
        classifier: The classifier model to be used (not utilized in the pipeline itself).

    Returns:
        loan_pipe: A scikit-learn Pipeline object that applies the preprocessing steps sequentially.
    """
    loan_pipe = Pipeline(
    [
        ('Arreglar mojibakes', pp.ArreglaMojibake(config.TEXT_COLUMNS)),
        ("Minúsculas y quitar stopwords", pp.QuitaStopwords(config.TEXT_COLUMNS)),
        ("Guardar en una columna", pp.JuntarFeatures(config.TEXT_COLUMNS, config.NEW_COLUMN)),
        ("Quitar features no deseadas", pp.DropFeatures(config.TEXT_COLUMNS))
    ]
    )
    return loan_pipe

In [37]:
X1_train = preproccess_pipeline().fit_transform(X1_train)
X2_train = preproccess_pipeline().fit_transform(X2_train)
X3_train = preproccess_pipeline().fit_transform(X3_train)

In [39]:
X1_train

,Texto_Limpio
126252,bonito excelente lugar travajo llenaron mmis e...
86765,casa campo excepcional ambiente relajarse disf...
87166,hermosa playa bebidas baratas preciosa playa p...
120252,calma llegamos dicen escalones parecen sencill...
154695,mejores tacos isla mujeres tacos riquísimos lu...
...,...
51457,gran lugar si gusta naturaleza aficionado ento...
136488,excelente sabor servicio deliciosa comida crem...
152136,cena lolos divino disfrutamos magníficas cenas...
136042,año nuevo nuevo menú encanta restaurante sabe ...


In [41]:
df1 = pd.concat([X1_train, y1_train], axis=1)
df2 = pd.concat([X2_train, y2_train], axis=1)
df3 = pd.concat([X3_train, y3_train], axis=1)
df3

,Texto_Limpio,Type
52557,tan maravilloso juego disneyland llegar azulik...,Hotel
706,visitar isla janitzio maravilloso ir descubrir...,Attractive
75004,desayuno estupendo fruta tortitas huevos ofrec...,Restaurant
6629,excelente cada vez visita obligada cada vez va...,Restaurant
200700,hotel agradable excelente ubicación desayuno g...,Hotel
...,...,...
77286,totalmente perdido disfrutamos así dirigimos c...,Restaurant
50959,normal ser número tripadvisor esperamos cosa m...,Restaurant
7066,mejor mexicana cenote simplemente increíble he...,Attractive
145830,sueños realidad resort incluido simplemente in...,Hotel


In [ ]:
# Vectorizer TFIDF
arg.tfidf_max_features = 2500
arg.tfidf_ngram_range = (1, 2)
arg.token_pattern=r'(?u)\b[^\d\W]+\b'

tfidf_vectorizer = TfidfVectorizer(max_features=arg.tfidf_max_features, 
                                   ngram_range=arg.tfidf_ngram_range, 
                                   token_pattern=arg.token_pattern)


### Cargamos en un Dataset

In [43]:
class restaurantDataset(Dataset):
    def __init__(self, data : pd.DataFrame, vectorizer, label_encoder,
                 y : str, standardize = False):
        """
        Initializes the restaurantDataset class.

        Args:
            data (pd.DataFrame): The input dataframe containing the text and labels.
            vectorizer (TfidfVectorizer): The TF-IDF vectorizer to transform text data.
            label_encoder (LabelEncoder): The label encoder to transform labels into numerical format.
            y (str): The column name in the dataframe representing the target labels.
            use_pca (bool, optional): Whether to apply PCA for dimensionality reduction. Default is False.
            pca (PCA, optional): The PCA object to use for dimensionality reduction. Required if use_pca is True.
            n_components (int, optional): The number of principal components to retain if PCA is applied. Default is 3.

        Attributes:
            data (pd.DataFrame): The input dataframe.
            n_samples (int): The number of samples in the dataset.
            X (torch.Tensor): The transformed feature matrix (TF-IDF or PCA-reduced).
            y (torch.Tensor): The transformed target labels.
            fitted_pca (PCA or None): The fitted PCA object if PCA is applied, otherwise None.

        Returns:
            None
        """
        self.data = data
        self.n_samples = len(data)
        
        # Transform text to TF-IDF vectors
        tfidf_matrix = vectorizer.transform(data[config.NEW_COLUMN])
        self.X = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float32)

        if standardize:
            scaler = StandardScaler()
            self.X_np = self.X.numpy()
            self.X = torch.tensor(scaler.fit_transform(self.X_np), dtype=torch.float32)
        
        # Transform labels to numbers
        self.y = torch.tensor(label_encoder.fit_transform(data[y]),
                              dtype=torch.long)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [46]:
arg.standardize = True
arg.y = "Type"

tfidf_vectorizer.fit(df3[config.NEW_COLUMN])

dataset = restaurantDataset(df3, tfidf_vectorizer, LabelEncoder(), arg.y,
                            arg.standardize)

### DataLoader

In [49]:
# Separamos en conjuntos de entrenamiento y de prueba.
arg.random_state = 42
X_train, X_test, y_train, y_test = train_test_split(dataset.X, dataset.y, 
                                                    test_size=0.2, 
                                                    random_state=arg.random_state,
                                                    stratify=dataset.y)

train_data = torch.utils.data.TensorDataset(X_train, y_train)
test_data = torch.utils.data.TensorDataset(X_test, y_test)

print("Longitud del train set", len(train_data))
print("Longitud del test set", len(test_data))

# Dataloaders
arg.batch_size = 128
train_dataloader = DataLoader(train_data, batch_size=arg.batch_size)
test_dataloader = DataLoader(test_data, batch_size=arg.batch_size)

for i, (X, y) in enumerate(test_dataloader):
    print(f"Shape of X [N, C]: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Longitud del train set 133152
Longitud del test set 33288
Shape of X [N, C]: torch.Size([128, 2500]) torch.float32
Shape of y: torch.Size([128]) torch.int64


### Primer modelo - Red Neuronal

In [51]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class TfidfClassifier(nn.Module):
    def __init__(self, input_size, output_size=3, dropout=0.3):
        super(TfidfClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(512, 128)
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(128, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        logits = self.fc3(x)
        return logits

    
arg.dropout = 0.2
arg.lr = 0.01
model = TfidfClassifier(input_size=2500, dropout=arg.dropout).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=arg.lr)

Using cuda device
TfidfClassifier(
  (fc1): Linear(in_features=2500, out_features=512, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
)


In [66]:
def train(dataloader, model, loss_fn, optimizer):
    """
    Entrena el modelo con los datos del dataloader y actualiza los pesos del modelo.
    
    Inputs:
    - dataloader: DataLoader con los datos de entrenamiento.
    - model: Modelo a entrenar.
    - loss_fn: Función de pérdida.
    - optimizer: Optimizador.
    """
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [59]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return 100*correct

In [60]:
def save_checkpoint(state, is_best, checkpoint_path, filename="checkpoint.pth",
                    best_filename="model_best.pth"):
    """
    Save the model checkpoint to the specified path.

    Args:
        state (dict): The state of the model to save, typically includes model weights and optimizer state.
        is_best (bool): If True, saves a copy of the checkpoint as "model_best.pth".
        checkpoint_path (str): The directory where the checkpoint will be saved.
        filename (str): The name of the checkpoint file. Default is "checkpoint.pth".

    Returns:
        None
    """
    filepath = os.path.join(checkpoint_path, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint_path, best_filename))

In [61]:
# Training hyperparameters
arg.lr = 1e-2
arg.epochs = 100
arg.patience = 20

# Scheduler hyperparameters
arg.lr_patience = 10
arg.lr_factor = 0.5 # Se reduce el learning rate a la mitad cada 10 epochs
# sin mejorar el desempeño

# Saving directory
arg.savedir = "../model"
os.makedirs(arg.savedir, exist_ok=True)

# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode = "max",
    patience=arg.lr_patience,
    factor=arg.lr_factor,
    verbose=True
)

c:\Users\Gus\miniconda3\envs\pytorch_env\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [68]:
epochs = 1000
best_metric = 0
n_no_improve = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, criterion, optimizer)
    tuning_metric = test(test_dataloader, model, criterion)
    
    # Update the scheduler
    scheduler.step(tuning_metric)
    
    # Save model checkpoint
    is_best = tuning_metric > best_metric
    if is_best:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve += 1
    
    save_checkpoint({
        "epoch": t + 1,
        "state_dict": model.state_dict(),
        "best_metric": best_metric,
        "optimizer": optimizer.state_dict(),
    }, is_best, arg.savedir, 
                    filename="checkpoint_on_2500_tfidf.pth", 
                    best_filename="model-best_on_2500_tfidf.pth")
    
    if n_no_improve >= arg.patience:
        print("No improvement. Breaking out of loop.")
        break
    
print("Ya quedó.")

Epoch 1
-------------------------------
loss: 0.030594  [  128/133152]
loss: 0.037685  [12928/133152]
loss: 0.027160  [25728/133152]
loss: 0.036106  [38528/133152]
loss: 0.040523  [51328/133152]
loss: 0.047097  [64128/133152]
loss: 0.029925  [76928/133152]
loss: 0.007163  [89728/133152]
loss: 0.026031  [102528/133152]
loss: 0.035927  [115328/133152]
loss: 0.057431  [128128/133152]
Test Error: 
 Accuracy: 94.0%, Avg loss: 1.883911 

Epoch 2
-------------------------------
loss: 0.030018  [  128/133152]
loss: 0.045759  [12928/133152]
loss: 0.017602  [25728/133152]
loss: 0.021486  [38528/133152]
loss: 0.042737  [51328/133152]
loss: 0.031869  [64128/133152]
loss: 0.022605  [76928/133152]
loss: 0.012631  [89728/133152]
loss: 0.045446  [102528/133152]
loss: 0.027482  [115328/133152]
loss: 0.059909  [128128/133152]
Test Error: 
 Accuracy: 94.0%, Avg loss: 1.885980 

Epoch 3
-------------------------------
loss: 0.054841  [  128/133152]
loss: 0.041730  [12928/133152]
loss: 0.018886  [25728/133